In [67]:
#Import packages
import pandas as pd
import numpy as np
import alminer
from astropy.coordinates import SkyCoord
from astropy import units as u

In [47]:
#Read in the derived data from Towner et al. 2023 IMF data
derived_data = pd.read_csv("/net/vdesk/data2/bach1/ballieux/master_project_2/data/high_mass_data/TableD1_full_sample_derived_properties_Towner_2023.ecsv",  comment='#', delimiter=' ')
print(derived_data.columns)
#The derived properties I trust, matches the numbers from the paper

#I also want to read in the observational properties
obs_data = pd.read_csv("/net/vdesk/data2/bach1/ballieux/master_project_2/data/high_mass_data/Table3_full_sample_observed_properties.ecsv",  comment='#', delimiter=' ')
print(obs_data.columns)
#TODO: this does not match the number of the paper, so figure out what is wrong. why? There are 315 candidates and 355 datapoints... not sure what that means. How can there be more datapoints?
#Either a mistake in reading it in, or some of the datapoints are not candidates. I think the latter, filter on only candidates. Not sure how this works, need to fix it 


Index(['Field', 'Outflow_ID', 'Classification', 'Ncol_blue_median',
       'Ncol_blue_MAD', 'Ncol_red_median', 'Ncol_red_MAD', 'M_blue',
       'M_blue_sigma_low', 'M_blue_sigma_upp', 'M_red', 'M_red_sigma_low',
       'M_red_sigma_upp', 'M_tot', 'M_tot_sigma_low', 'M_tot_sigma_upp',
       'P_blue', 'P_blue_sigma_low', 'P_blue_sigma_upp', 'P_red',
       'P_red_sigma_low', 'P_red_sigma_upp', 'P_tot', 'P_tot_sigma_low',
       'P_tot_sigma_upp', 'E_blue', 'E_blue_sigma_low', 'E_blue_sigma_upp',
       'E_red', 'E_red_sigma_low', 'E_red_sigma_upp', 'E_tot',
       'E_tot_sigma_low', 'E_tot_sigma_upp', 't_dyn', 't_dyn_err', 'Mdot_blue',
       'Mdot_blue_sigma_low', 'Mdot_blue_sigma_upp', 'Mdot_red',
       'Mdot_red_sigma_low', 'Mdot_red_sigma_upp', 'Mdot_tot',
       'Mdot_tot_sigma_low', 'Mdot_tot_sigma_upp', 'Pdot_blue',
       'Pdot_blue_sigma_low', 'Pdot_blue_sigma_upp', 'Pdot_red',
       'Pdot_red_sigma_low', 'Pdot_red_sigma_upp', 'Pdot_tot',
       'Pdot_tot_sigma_low', 'Pdot_to

In [87]:
#Step 1: we are only interested in the likely sources
likely_dataframe_derived = derived_data[derived_data['Classification']=='likely']
print('There are this many possible outflows with the classification likely:', len(likely_dataframe_derived['Classification']))

#Again, the derived ones is correct but the observed one not

likely_dataframe_obs = obs_data[obs_data['Classification']=='likely']
print('There are this many possible outflows with the classification likely:', len(likely_dataframe_obs['Classification']))

There are this many possible outflows with the classification likely: 129
There are this many possible outflows with the classification likely: 149


In [86]:
#Step2: here we start the process of querying the sources. First define the coordinates and some things required to run the query
coordinates_obs=SkyCoord(likely_dataframe_obs["RA"], likely_dataframe_obs["Dec"] , unit=(u.hourangle, u.deg))

query_catalog_obs =  {"Name": likely_dataframe_obs["Field"],
          "RAJ2000" : coordinates.ra.deg,
          "DEJ2000" : coordinates.dec.deg}

query_catalog_obs = pd.DataFrame(query_catalog_obs)

#TODO: fix the right radius, etc...
#Here we run the query
ALMA_IMF_query = alminer.catalog(query_catalog_obs)

alminer.catalog results
Target = G008.67
--------------------------------
Number of projects = 6
Number of observations = 21
Number of unique subbands = 76
Total number of subbands = 116
6 target(s) with ALMA data = ['G008.67', 'G08.68-0.37', 'G08.670-00.356', 'G008.6702-0.3557', 'G008.6834-0.3675', 'flow01']
--------------------------------
Target = G008.67


KeyboardInterrupt: 